In [1]:
import numpy as np
import time
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure 
from bokeh.layouts import row, column
from bokeh.models import Label
from ipywidgets import interact
output_notebook()

Loading BokehJS ...

In [27]:
# Initializing arrays for plotting
timepoints = np.zeros(timesteps)
poscar1 = np.zeros(timesteps)
poscar1_est = np.zeros(timesteps)
poscar1_meas = np.zeros(timesteps)
poscar2 = np.zeros(timesteps)
car_sep = np.zeros(timesteps)
car2_acc = np.zeros(timesteps)
poscar2_kf = np.zeros(timesteps)
car_sep_kf = np.zeros(timesteps)
car2_acc_kf = np.zeros(timesteps)
noiserange = np.zeros(timesteps)
pdf_proc = np.zeros(timesteps)
pdf_meas = np.zeros(timesteps)
def InitPlotArrays():    
    timepoints.fill(np.nan)
    poscar1.fill(np.nan)
    poscar1_est.fill(np.nan)
    poscar1_meas.fill(np.nan)
    poscar2.fill(np.nan)
    car_sep.fill(np.nan)
    car2_acc.fill(np.nan)
    poscar2_kf.fill(np.nan)
    car_sep_kf.fill(np.nan)
    car2_acc_kf.fill(np.nan)
    noiserange.fill(np.nan)
    pdf_proc.fill(np.nan)
    pdf_meas.fill(np.nan)

In [28]:
# Setup plots
InitPlotArrays()

# Adding legends
p6 = figure(plot_width = 400, plot_height = 120, x_range=[-0.5, 20], y_range=[-1, 5])
colors = ['darkslategray', 'gold', 'crimson']
p6.rect([0.2, 0.2, 0.2], [4, 2, 0], width=1, height=1, color=colors, 
              width_units='data', height_units='data')
label1 = Label(x=1, y=3.3, x_units='data', y_units='data', text='Leading Car')
label2 = Label(x=1, y=1.3, x_units='data', y_units='data', text='Following Car')
label3 = Label(x=1, y=-0.7, x_units='data', y_units='data', text='Following Car with Kalman Filter')
p6.add_layout(label1)
p6.add_layout(label2)
p6.add_layout(label3)
p6.yaxis.visible = False
p6.ygrid.visible = False
p6.xaxis.visible = False
p6.xgrid.visible = False
p6.toolbar.logo = None
p6.toolbar_location = None

# Figure 1
p5 = figure(plot_width = 400, plot_height = 200, title='Noise Distributions')
r11 = p5.line(noiserange, pdf_proc, line_color='seagreen', line_width=4, legend='Process')
r12 = p5.line(noiserange, pdf_meas, line_color='greenyellow', line_width=4, legend='Measured')
p5.toolbar.logo = None
p5.toolbar_location = None
p5.legend.location = 'bottom_center'

# Figure 2
p4 = figure(plot_width=1000, plot_height=150, x_axis_label='position', title='Car Following Animation',
            x_range=[0, dist_size], y_range=[-0.4, 1.0])
colors = ['darkslategray', 'gold', 'crimson']
r10 = p4.rect(x=[car1_initpos, car2_initpos, car2_initpos], y=[0.0, -0.2, 0.2], 
              width=car_size, height=8, color=colors, 
              width_units='data', height_units='screen')
p4.yaxis.visible = False
p4.ygrid.visible = False
p4.toolbar.logo = None
p4.toolbar_location = None

# Figure 3
p1 = figure(plot_width=500, plot_height=400, x_axis_label='time', y_axis_label='position',
            title='Car Positions', x_range=[0, timesteps * dt], y_range=[0, dist_size])
r1 = p1.line(timepoints, poscar1, color='darkslategray', line_width=4)
r2 = p1.line(timepoints, poscar1_meas, color='royalblue', 
             legend='Measured position', line_width=2)
r3 = p1.line(timepoints, poscar1_est, color='skyblue', 
             legend='KF estimated position', line_width=2)
r4 = p1.line(timepoints, poscar2, color='gold', line_width=4)
r5 = p1.line(timepoints, poscar2_kf, color='crimson', line_width=4)
p1.legend.location = 'bottom_right'
p1.toolbar.logo = None
p1.toolbar_location = None

# Figure 2
p2 = figure(plot_width=500, plot_height=200, x_axis_label='time', y_axis_label='separation',
            title='Car Separations', x_range=[0, timesteps * dt], y_range=[0, 4 * dist_sep])
r6 = p2.line(timepoints, car_sep, color='gold', line_width=4)
r7 = p2.line(timepoints, car_sep_kf, color='crimson', line_width=4)
p2.toolbar.logo = None
p2.toolbar_location = None

# Figure 3
p3 = figure(plot_width=500, plot_height=200, x_axis_label='time', y_axis_label='acceleration',
            title='Following Car Acceleration', x_range=[0, timesteps * dt], y_range=[-car2_maxbrake, car2_maxacc])
r8 = p3.line(timepoints, car2_acc, color='gold', line_width=4)
r9 = p3.line(timepoints, car2_acc_kf, color='crimson', line_width=4)
p3.toolbar.logo = None
p3.toolbar_location = None

In [29]:
def RunCarFollowing():
    
    # Setup initial values
    InitPlotArrays()
    pos1real = car1_initpos
    pos2 = car2_initpos
    pos2_kf = car2_initpos
    vel2 = car2_initvel
    vel2_kf = car2_initvel
    acc2 = car2_initacc
    acc2_kf = car2_initacc
    tt = 0
    realt = 0
    
    # Update noise distributions
    if noise_type == 'gauss':
        nlim = max(process_noise_level, measurement_noise_level)
        global noiserange, pdf_proc, pdf_meas
        noiserange = np.linspace(-4 * nlim, 4 * nlim, timesteps)
        pdf_proc = (1 / (process_noise_level * np.sqrt(2 * np.pi)) 
                    * np.exp(-noiserange**2 / (2 * process_noise_level**2)))
        r11.data_source.data['x'] = noiserange
        r11.data_source.data['y'] = pdf_proc
        pdf_meas = (1 / (measurement_noise_level * np.sqrt(2 * np.pi)) 
                    * np.exp(-noiserange**2 / (2 * measurement_noise_level**2)))
        r12.data_source.data['x'] = noiserange
        r12.data_source.data['y'] = pdf_meas
        push_notebook(handle=target3)
    
    # Initialize Kalman Filter
    f = KalmanFilter(dim_x=2, dim_z=1)
    f.x = np.array([[car1_initpos],  # position
                    [car1_vel]])  # velocity
    f.F = np.array([[1.0, 1.0],
                    [0.0, 1.0]])
    f.H = np.array([[1.0, 0.0]])
    f.P = np.array([[1000.0, 0.0],
                    [0.0, 1000.0]])
    f.R = np.array([[5.0]])
    f.Q = Q_discrete_white_noise(dim=2, dt=dt, var=process_noise_level**2)  # Process noise
    
    # Loop for updating positions
    for tt in range(timesteps):

        # Real position of leading car
        pos1real = pos1real + car1_vel * dt

        # Measurement of position of leading car with noise
        if noise_type == 'gauss':
            z = np.random.normal(pos1real, measurement_noise_level)
        elif noise_type == 'power':
            z = (pos1real + 2.0 * measurement_noise_level 
                 * np.sign((np.random.uniform() - 0.5)) * np.random.power(5))
        elif noise_type == 'uniform':
            z = pos1real + 4.0 * measurement_noise_level * (np.random.uniform() - 0.5)
        else:
            z = pos1real

        # Kalman Filter predict and update steps
        f.predict()
        f.update(z)   

        # Following car update without Kalman Filter
        dx = z - pos2
        err = (dx - dist_sep) / dist_sep
        if err > 1.0: err = 1.0
        # Acceleration model
        if err > 0:
            acc2 = err * car2_maxacc
        else:
            acc2 = err * car2_maxbrake
        vel2 = vel2 + acc2 * dt
        # crash protection 
        if vel2 < 0:
            vel2 = 0
            acc2 = 0
        # update position
        pos2 = pos2 + vel2 * dt + 0.5 * acc2 * dt**2

        # Following car update with Kalman Filter
        dx_kf = f.x[0, 0] - pos2_kf
        err_kf = (dx_kf - dist_sep) / dist_sep
        if err_kf > 1.0: err_kf = 1.0
        # Acceleration model
        if err_kf > 0:
            acc2_kf = err_kf * car2_maxacc
        else:
            acc2_kf = err_kf * car2_maxbrake
        vel2_kf = vel2_kf + acc2_kf * dt
        # crash protection
        if vel2_kf < 0:
            vel2_kf = 0
            acc2_kf = 0
        # update position
        pos2_kf = pos2_kf + vel2_kf * dt + 0.5 * acc2_kf * dt**2

        # Update arrays for plotting
        global timepoints, poscar1, poscar2, poscar1_est, car_sep
        timepoints[tt] = tt * dt
        poscar1[tt] = pos1real
        poscar1_est[tt] = f.x[0, 0]
        poscar1_meas[tt] = z
        poscar2[tt] = pos2
        poscar2_kf[tt] = pos2_kf
        car_sep[tt] = pos1real - pos2
        car_sep_kf[tt] = pos1real - pos2_kf  
        car2_acc[tt] = acc2
        car2_acc_kf[tt] = acc2_kf

        # Update plots
        #first row
        r10.data_source.data['x'] = [pos1real, pos2, pos2_kf]
        push_notebook(handle=target2)
        #second row
        r1.data_source.data['x'] = timepoints
        r1.data_source.data['y'] = poscar1
        r2.data_source.data['x'] = timepoints
        r2.data_source.data['y'] = poscar1_meas
        r3.data_source.data['x'] = timepoints
        r3.data_source.data['y'] = poscar1_est
        r4.data_source.data['x'] = timepoints
        r4.data_source.data['y'] = poscar2
        r5.data_source.data['x'] = timepoints
        r5.data_source.data['y'] = poscar2_kf
        r6.data_source.data['x'] = timepoints
        r6.data_source.data['y'] = car_sep
        r7.data_source.data['x'] = timepoints
        r7.data_source.data['y'] = car_sep_kf
        r8.data_source.data['x'] = timepoints
        r8.data_source.data['y'] = car2_acc
        r9.data_source.data['x'] = timepoints
        r9.data_source.data['y'] = car2_acc_kf
        push_notebook(handle=target1)

        time.sleep(0.1)

### Change the initial values in the cell below to simulate different conditions

In [36]:
# Initial values
car1_initpos = 3  # starting position of the car in the front
car2_initpos = 0  # starting position of the following cars (keep at 0)
car1_vel = 4  # velocity of the car in the front
car2_initvel = 4  # starting velocity of the following cars
car_size = 1  # size of the car 
dist_sep = 3 * car_size  # desired ideal separation between the car in the front and the cars following
dist_size = 20  # length of the track in arb. units
car1_acc = 0  # accelaration of the car in the front (keep at 0)
car2_initacc = 0 # starting accelaration of the following cars
car2_maxacc = 2  # maximum possible accelaration for the following cars
car2_maxbrake = 40  # maximum possible decelaration for the following cars
noise_type = 'gauss'  # type of measurement noise (can use 'gauss', 'power', and 'uniform', if anything else is entered measurement noise will be 0
measurement_noise_level = 0.5  # use a number between 0.1 and 1.5
process_noise_level = 0.5  # use a number between 0.1 and 1.5
dt = 0.015  # time step (keep at 0.015)
timesteps = int((dist_size - car1_initpos) / car1_vel / dt)  # total number of timesteps 

In [37]:
target3 = show(row(p5, p6), notebook_handle=True)
target2 = show(p4, notebook_handle=True)
target1 = show(row(p1, column(p2, p3)), notebook_handle=True)
RunCarFollowing()